In [ ]:
import requests

market = "BTCUSDT"
tick_interval = "1h"

url = f"https://api.binance.com/api/v3/klines?symbol={market}&interval={tick_interval}"
data = requests.get(url).json()


In [ ]:
from binance.client import Client
import pandas as pd
from datetime import timedelta, datetime

client = Client()
from_date = datetime.now() - timedelta(days=600)
candles = client.get_historical_klines(
    symbol="BTCUSDT",
    interval=Client.KLINE_INTERVAL_1DAY,
    start_str=from_date.isoformat(),
)

cd = pd.DataFrame(
    candles,
    columns=[
        "Date",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Can be ignored",
    ],
    dtype=float,
)

for i in cd.index:
    cd.at[i, "Date"] = datetime.fromtimestamp(cd.at[i, "Date"] / 1000)

cd.index = pd.DatetimeIndex(cd["Date"])
cd.head(3)


In [ ]:
import mplfinance as mpf
from datetime import datetime

chart_data = cd[["Date", "Open", "High", "Low", "Close", "Volume"]]

mpf.plot(chart_data.tail(50), type="candle")


In [ ]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA


class SmaCross(SignalStrategy, TrailingStrategy):
    n1 = 20
    n2 = 50

    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()

        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)

        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only

        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * 0.95

        # Set order entry sizes using the method provided by
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)

        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)


from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(chart_data, SmaCross, cash=100000, commission=0.002)

bt.run()
bt.plot(filename="../src/results/SmaCross")


In [ ]:
from talib import ADX, RSI, EMA, SMA
import matplotlib.pyplot as plt

rsi = RSI(chart_data["Close"])
rsiMA = SMA(rsi)

adx = ADX(chart_data["High"], chart_data["Low"], chart_data["Close"])
adxMA = SMA(adx)

ema = EMA(chart_data["Close"], timeperiod=200)

figure, axis = plt.subplots(3, 1, figsize=(15, 5))

axis[0].set_title("EMA")
axis[0].plot(ema)

axis[1].set_title("RSI")
axis[1].plot(rsi)
axis[1].plot(rsiMA)

axis[2].set_title("ADX")
axis[2].plot(adx)
axis[2].plot(adxMA)


# plt.figure(figsize=(15,5))
# plt.title('RSI')
# plt.plot(rsi)

# plt.title('ADX')
# plt.plot(adx)


In [ ]:
indicatorsDF = pd.DataFrame(rsi, columns=["rsi"])
emaDF = pd.DataFrame(ema, columns=["ema"])
adxDF = pd.DataFrame(adx, columns=["adx"])
indicatorsDF.merge(adxDF, right_index=True, left_index=True).merge(
    emaDF, right_index=True, left_index=True
)


In [ ]:
import ccxt

exchange = ccxt.binanceus({})
bars = exchange.fetch_ohlcv("ETH/USDT", timeframe="1m", limit=400)
print(bars)
